In [14]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
import gensim
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools
import nltk
from nltk import bigrams
from nltk.util import ngrams
from nltk import word_tokenize, pos_tag, pos_tag_sents
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
colnames=['A', 'B', 'C', 'D','E','text'] 
df = pd.read_csv("short.csv", encoding ="ISO-8859-1", names=colnames, header=None)
df.drop(['A', 'B', 'C', 'D','E'], axis=1, inplace=True)
print(df)
df.head()

                                                    text
0      @switchfoot http://twitpic.com/2y1zl - Awww, t...
1      is upset that he can't update his Facebook by ...
2      @Kenichan I dived many times for the ball. Man...
3        my whole body feels itchy and like its on fire 
4      @nationwideclass no, it's not behaving at all....
...                                                  ...
99995  my son has developed the new habit of waking u...
99996  looks like my routers broke  more tweets from ...
99997  i really dont want to be in college right now....
99998                        @flossa  *offers you pepto*
99999  @JosieHobo I WOULD SOOOOO BE THERE IF I DIDN'T...

[100000 rows x 1 columns]


,text
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,is upset that he can't update his Facebook by ...
2,@Kenichan I dived many times for the ball. Man...
3,my whole body feels itchy and like its on fire
4,"@nationwideclass no, it's not behaving at all...."


In [4]:
import pandas as pd
import numpy as np
hash_list = []
at_list = []
for index, row in df.iterrows():
  temp_list = [x.group() for x in re.finditer( r'@\w*', row.item())]
  at_list.extend(temp_list)

for index, row in df.iterrows():
  temp_list = [x.group() for x in re.finditer( r'#\w*', row.item())]
  hash_list.extend(temp_list)

print(hash_list)
print(at_list)

['#itm', '#therapyfail', '#fb', '#TTSC', '#24', '#gayforpeavy', '#FML', '#3', '#camerafail', '#', '#', '#fb', '#', '#travian', '#wow', '#art', '#fail', '#KrispyKreme', '#fb', '#24', '#the', '#twitter', '#fail', '#CNN', '#1', '#Kulula', '#surface', '#c25k', '#drupal', '#epicfail', '#rails', '#mhbigcatch', '#stuckrecord', '#twitteriffic', '#startrek', '#earthquake', '#php', '#poken', '#error', '#tracy', '#fb', '#dwsr', '#Italy', '#Earthquake', '#fantasysurfer', '#titsuptuesday', '#N85', '#heyxboxlive', '#House', '#babypunch', '#cob', '#ticket', '#fb', '#fb', '#terremoto', '#earthquake', '#italy', '#fail', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#itm', '#AutomationAtaCost', '#gsoc', '#rockbox', '#niceweather', '#Berlin', '#goodlife', '#Ambients', '#theinbetweeners', '#Twilight', '#Summers', '#mmuk09', '#i36', '#spotify', '#42', '#fb', '#pman', '#spbpuk', '#dreams', '#dreams', '#dwsr', '#', '#', '#', '#fb', '#wolframalpha', '#search', '#semantic', '#Virgin', '#liquor',

In [6]:
#mylist = [x.group() for x in re.finditer( r'@\w*', mystring)]
r1 = [x for x, _ in Counter(hash_list).most_common(20)]
print("top 20 #s are: ",r1)
#mylist = [x.group() for x in re.finditer( r'@\w*', mystring)]
r2 = [x for x, _ in Counter(at_list).most_common(20)]
print("top 20 @s are: ",r2)

top 20 #s are:  ['#asot400', '#', '#fb', '#f1', '#fail', '#ASOT400', '#1', '#tag', '#F1', '#2', '#NAME', '#awaresg', '#fixreplies', '#pawpawty', '#swineflu', '#bck5', '#hoppusday', '#ipl', '#readathon', '#followfriday']
top 20 @s are:  ['@', '@mileycyrus', '@tommcfly', '@ddlovato', '@DavidArchie', '@stephenfry', '@petewentz', '@DonnieWahlberg', '@JonathanRKnight', '@nick_carter', '@joeymcintyre', '@jordanknight', '@aplusk', '@dougiemcfly', '@amazingphoebe', '@heidimontag', '@JoelMadden', '@shaundiviney', '@dannywood', '@gfalcone601']


In [19]:
tokenize_1 = nltk.word_tokenize(my_string)
bigrams = ngrams(tokenize_1,2)
trigrams=ngrams(tokenize,3)
fourgrams=ngrams(tokenize,4)
r3 = [x for x, _ in Counter(tokenize_1).most_common(20)]
r4 = [x for x, _ in Counter(bigrams).most_common(20)]
r5 = [x for x, _ in Counter(trigrams).most_common(20)]
r6 = [x for x, _ in Counter(fourgrams).most_common(20)]
print("top 20 1-gram are: ",r3)
print("top 20 2-gram are: ",r4)
print("top 20 3-gram are: ",r5)
print("top 20 4-gram are: ",r6)

top 20 1-gram are:  ['.', '!', 'I', 'to', '@', 'the', ',', 'i', 'a', '...', 'my', 'and', 'it', 'is', "n't", '?', 'in', 'for', 'of', "'s"]
top 20 2-gram are:  [('!', '!'), ('I', "'m"), ('.', 'I'), ('.', '@'), ('!', '@'), ('do', "n't"), ('!', 'I'), ('ca', "n't"), ('in', 'the'), ('&', 'quot'), ('quot', ';'), ('it', "'s"), ('?', '?'), ('http', ':'), ('to', 'go'), ('I', 'have'), (',', 'I'), ('going', 'to'), ('&', 'amp'), ('amp', ';')]
top 20 3-gram are:  [('!', '!', '!'), ('&', 'quot', ';'), ('&', 'amp', ';'), ('I', 'do', "n't"), ('?', '?', '?'), ('I', 'ca', "n't"), ('to', 'go', 'to'), ('.', 'I', "'m"), ('!', '!', '@'), ('&', 'lt', ';'), ('!', '!', 'I'), ('I', 'have', 'to'), ('I', 'did', "n't"), ('!', 'I', "'m"), ('I', 'wish', 'I'), ('want', 'to', 'go'), ('.', 'It', "'s"), ("n't", 'want', 'to'), ('I', "'m", 'not'), ('do', "n't", 'want')]
top 20 4-gram are:  [('!', '!', '!', '!'), ('?', '?', '?', '?'), ('!', '!', '!', '@'), ('&', 'lt', ';', '3'), ('!', '!', '!', 'I'), ('do', "n't", 'want', '

In [48]:
import pandas as pd
import string
from nltk import WordPunctTokenizer, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn, stopwords

# Tokenize text
tokenizer = nltk.WordPunctTokenizer()

df["Tokens"] = df["text"].str.lower().apply(tokenizer.tokenize)


# Remove punctuations
df['text'] = df['text'].apply(lambda x:''.join([i for i in x 
                                                  if i not in string.punctuation]))

# Remove stopwords
stopwords = stopwords.words('english')

def remove_stopwords(tokens):
    filtered_words = [word for word in tokens if word not in stopwords]
    return filtered_words

df["Tokens"] = df["Tokens"].apply(remove_stopwords)
df["Tokens"] 

0        [switchfoot, httptwitpiccom2y1zl, awww, thats,...
1        [upset, cant, update, facebook, texting, might...
2        [kenichan, dived, many, times, ball, managed, ...
3                  [whole, body, feels, itchy, like, fire]
4          [nationwideclass, behaving, im, mad, cant, see]
                               ...                        
99995    [son, developed, new, habit, waking, 530am, im...
99996          [looks, like, routers, broke, tweets, fone]
99997    [really, dont, want, college, right, wish, sunny]
99998                              [flossa, offers, pepto]
99999          [josiehobo, would, sooooo, didnt, revision]
Name: Tokens, Length: 100000, dtype: object

In [61]:
df['POS'] = pos_tag_sents( df['text'].apply(word_tokenize).tolist() )
print(df['POS'])

0        [(switchfoot, NN), (httptwitpiccom2y1zl, NN), ...
1        [(is, VBZ), (upset, JJ), (that, IN), (he, PRP)...
2        [(Kenichan, NNP), (I, PRP), (dived, VBD), (man...
3        [(my, PRP$), (whole, JJ), (body, NN), (feels, ...
4        [(nationwideclass, NN), (no, DT), (its, PRP$),...
                               ...                        
99995    [(my, PRP$), (son, NN), (has, VBZ), (developed...
99996    [(looks, NNS), (like, IN), (my, PRP$), (router...
99997    [(i, NN), (really, RB), (dont, JJ), (want, VBP...
99998    [(flossa, NN), (offers, NNS), (you, PRP), (pep...
99999    [(JosieHobo, NN), (I, PRP), (WOULD, VBP), (SOO...
Name: POS, Length: 100000, dtype: object


In [62]:
noun = []
for  index, row in df.iterrows():
  noun.append([word for word,pos in pos_tag(row[0]) if pos == 'NN'])
print(noun)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

